In [418]:
import cfscrape
from lxml import etree
import datetime
import os.path

from random import randint
from time import sleep

import pandas as pd
import numpy as np

Based on https://productforums.google.com/forum/#!msg/webmasters/8fPg1I2p34w/Xsdw0stkDwAJ and https://moz.com/blog/the-ultimate-guide-to-the-google-search-parameters

In [419]:
df_urls=pd.DataFrame()

keywords = ['booter','stresser','ddoser']

for keyword in keywords:
    print(keyword)
    
    page_number=1
    attempts=0
    while attempts < 3:
        print('Page number: ',str(page_number))

        url="https://www.google.nl/search?q="+keyword+"&start="+str((page_number - 1)*10)
        scraper = cfscrape.create_scraper()
        scraped_html=scraper.get(url).content
        html_tree = etree.HTML(scraped_html)

        results_temp=html_tree.xpath("//h3/a/@href") #for the links

        if not results_temp:       
            attempts += 1
            print('\t*Empty')

        df_urls=df_urls.append(pd.DataFrame(results_temp, [keyword]*len(results_temp)))

        page_number+= 1
        sleep(randint(1,5))

booter
Page number:  1
Page number:  2
Page number:  3
Page number:  4
Page number:  5
Page number:  6
Page number:  7
Page number:  8
Page number:  9
Page number:  10
Page number:  11
Page number:  12
Page number:  13
Page number:  14
Page number:  15
Page number:  16
Page number:  17
Page number:  18
Page number:  19
Page number:  20
Page number:  21
Page number:  22
Page number:  23
Page number:  24
Page number:  25
	*Empty
Page number:  26
	*Empty
Page number:  27
Page number:  28
	*Empty
stresser
Page number:  1
Page number:  2
Page number:  3
Page number:  4
Page number:  5
Page number:  6
Page number:  7
Page number:  8
Page number:  9
Page number:  10
Page number:  11
Page number:  12
Page number:  13
Page number:  14
Page number:  15
	*Empty
Page number:  16
	*Empty
Page number:  17
	*Empty
ddoser
Page number:  1
Page number:  2
Page number:  3
Page number:  4
Page number:  5
Page number:  6
Page number:  7
Page number:  8
Page number:  9
Page number:  10
Page number:  11
Page

In [420]:
df_urls.reset_index(inplace=True)
len(df_urls)

575

In [421]:
# STRIP http(s) and www and leave only the domain name + TLD
df_urls['strip_url']=df_urls[0].str.split('//').str[1].\
                                str.split('/').str[0].\
                                str.replace('www.','')
        
# DROP duplicated 
df_remaining=df_urls.drop_duplicates(subset=['strip_url']).dropna()
len(df_remaining)

461

In [422]:
not_booter=['wikipedia',
            'webopedia',
            'wiktionary',
            'merriam-webster',
            'twitter',
            'reddit',
            'incapsula',
            'github', 
            'youtube',
            '.google',
            'urbandictionary',
            'arstechnica',
            'sourceforge',
            'aliexpress',
            'instagram',
            'arbornetworks',
            'radware',
            'damonmccoy',
            'jairsantanna',
            'krebsonsecurity',
            'booterblacklist.com',
            'hackforum',
            'facebook',
            'amazon',
            'tripadvisor',
            'linkedin',
            'spotify',
            'linguee',
            'glassdoor',
            'forum.',
            'blog.']

In [423]:
df_remaining=df_remaining[~df_remaining['strip_url'].str.contains('|'.join(not_booter))]['strip_url']
len(df_remaining)

395

In [471]:
df_groundtruth = pd.read_csv('20180215_groundtruth.csv')
len(df_groundtruth)

1035

## Creating a file of urls '2classify' (manually or automatically)

In [431]:
df_2classify=df_remaining[~df_remaining.isin(df_groundtruth['url'])]
df_2classify.to_csv('2classify.csv',  index = False)

## After classifying URLs, then merge with the current ground truth

In [474]:
import datetime
today=f"{datetime.datetime.now():%Y%m%d}"
today

'20180215'

In [ ]:
df_groundtruth=df_groundtruth.append(pd.read_csv ('2classify.csv',sep = ';'))
df_groundtruth.drop_duplicates().to_csv(today+'_groundtruth.csv', index = False)

!rm 2classify.csv

In [461]:
len(df_groundtruth)

1035

## Extracting the booterlist 

In [476]:
# NOT NEEDED!!
df_groundtruth=pd.read_csv(today+'_groundtruth.csv')

In [477]:
booterlist = df_groundtruth[df_groundtruth['booter?']== 'Y']['url'].drop_duplicates().sort_values()
booterlist.to_csv(today+'_booterlist.csv',  index = False)